In [86]:
import numpy as np
from sklearn import datasets
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix

In [87]:
def make_labelled(column):              #divinding the data into labels
    second_limit=column.mean() #------(first)---------(second)----------(third)----------#
    first_limit=0.5*second_limit
    third_limit=1.5*second_limit
    
    for i in range(len(column)):
        if column[i]<first_limit:
            column[i]=0
        elif column[i]<second_limit:
            column[i]=1
        elif column[i]<third_limit:
            column[i]=2
        else:
            column[i]=3
                
    return column

In [88]:
iris=datasets.load_iris()
X=iris.data
Y=iris.target

In [89]:
for j in range(0,X.shape[-1]):
    X[:,j]=make_labelled(X[:,j])

In [201]:
X=X.astype(int) #converting to float

In [202]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

In [203]:
def fit(x_train,y_train):
    result={}
    classes=set(y_train)
    for current_class in classes:
        result[current_class]={}  #for each class make a dictionary of each feature
        result['total_data']=len(y_train) #total number of data points of each class
        current_class_rows=y_train==current_class
        x_train_current=x_train[current_class_rows] 
        y_train_current=y_train[current_class_rows]
        num_feature=x_train_current.shape[1] #number of columns
        result[current_class]['total_count']=len(y_train_current) #count of the all datapoints of current class
        for j in range(1,num_feature+1): #number of features from 1..2..3.....n #making a dictionary of features
            result[current_class][j]={} #foreach(jth) feature make a dictionary of counts->of each label(distinct class)
            values=set(x_train[:,j-1])  #j-1 because actual mai toh index is from 0..1..2..n-1
            for value in values:
                result[current_class][j][value]=(x_train_current[:,j-1]==value).sum() #storing count of each label
                
    return result

In [204]:
def probability(dictionary,x,current_class):
    prob_current_class=dictionary[current_class]['total_count']/dictionary['total_data'] # P(y==ai)
    num_features=len(dictionary[current_class].keys())-1 #loop through each feature prob #-1 coz last row is count
    output=1
    for j in range(1,num_features+1): #from 1..2..3....n
        xj=x[j-1] #value of jth feature
        count_current_class_with_value_xj=dictionary[current_class][j][xj]+1 #find prob of the label present in the jth feature with laplace correction 
        count_current_class=dictionary[current_class]['total_count'] + len(dictionary[current_class][j].keys()) 
        current_xj_probability=count_current_class_with_value_xj/count_current_class
        output=output*current_xj_probability*prob_current_class # NAIVE BAYES HERE because we are mult prob of each feature given class=current_class = P(X=x)=P(X=x1)*P(X=x2)... 
        
    return output #this value can be very small when current_xj_probability is small and can reduce to zero when mult with other small probs. so we can use log whihc will lead to addition

In [205]:
def probability_using_log(dictionary,x,current_class):
    prob_current_class=np.log(dictionary[current_class]['total_count'])-np.log(dictionary['total_data']) # P(y==ai)
    num_features=len(dictionary[current_class].keys())-1; #loop through each feature prob #-1 coz last row is count
    output=0
    for j in range(1,num_features+1): #from 1..2..3....n
        xj=x[j-1] #value of jth feature
        print(xj)
        count_current_class_with_value_xj=dictionary[current_class][j][xj] + 1   #find prob of the label present in the jth feature with laplace correction 
        count_current_class=dictionary[current_class]['total_count'] + len(dictionary[current_class][j].keys()) 
        current_xj_probability=np.log(count_current_class_with_value_xj)-np.log(count_current_class)
        output=output + current_xj_probability + prob_current_class # NAIVE BAYES HERE because we are mult prob of each feature given class=current_class = P(X=x)=P(X=x1)*P(X=x2)... 
        
    return output #this value can be very small when current_xj_probability is small and can reduce to zero when mult with other small probs. so we can use log whihc will lead to addition

In [206]:
def predict_one_point(dictionary,x):
    classes=dictionary.keys()
    best_p=-1000
    best_class=-1
    first=True
    for current_class in classes: #find probabbilites of this feature row being current class
        if current_class=="total_data":
            continue
        p_current=probability_using_log(dictionary,x,current_class) #
        if first or p_current>best_p:
            best_p=p_current
            best_class=current_class
        first=False
    return best_class #whichever out of the prob is max predict that

In [207]:
def predict(dictionary,x_test):
    y_pred=[]
    for x in x_test:
        temp=predict_one_point(dictionary,x) #sending one class to for prediction 
        y_pred.append(temp)
    return y_pred

In [208]:
dictionary=fit(x_train,y_train)

In [209]:
dictionary

{0: {'total_count': 37,
  1: {1: 37, 2: 0},
  2: {1: 6, 2: 31},
  3: {0: 37, 1: 0, 2: 0, 3: 0},
  4: {0: 36, 1: 1, 2: 0, 3: 0}},
 'total_data': 112,
 1: {'total_count': 34,
  1: {1: 19, 2: 15},
  2: {1: 28, 2: 6},
  3: {0: 0, 1: 6, 2: 28, 3: 0},
  4: {0: 0, 1: 8, 2: 25, 3: 1}},
 2: {'total_count': 41,
  1: {1: 4, 2: 37},
  2: {1: 26, 2: 15},
  3: {0: 0, 1: 0, 2: 24, 3: 17},
  4: {0: 0, 1: 0, 2: 4, 3: 37}}}

In [210]:
y_predicted=predict(dictionary,x_test)

1
1
2
3
1
1
2
3
1
1
2
3
2
1
2
1
2
1
2
1
2
1
2
1
1
2
0
0
1
2
0
0
1
2
0
0
2
1
3
3
2
1
3
3
2
1
3
3
1
2
0
0
1
2
0
0
1
2
0
0
2
2
3
3
2
2
3
3
2
2
3
3
1
2
0
0
1
2
0
0
1
2
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
2
1
2
2
1
2
0
0
1
2
0
0
1
2
0
0
2
1
2
2
2
1
2
2
2
1
2
2
1
1
2
2
1
1
2
2
1
1
2
2
1
1
0
0
1
1
0
0
1
1
0
0
1
2
0
0
1
2
0
0
1
2
0
0
1
1
2
3
1
1
2
3
1
1
2
3
1
1
2
2
1
1
2
2
1
1
2
2
1
2
1
0
1
2
1
0
1
2
1
0
1
1
0
0
1
1
0
0
1
1
0
0
2
1
2
3
2
1
2
3
2
1
2
3
1
2
0
0
1
2
0
0
1
2
0
0
1
2
1
0
1
2
1
0
1
2
1
0
2
1
2
2
2
1
2
2
2
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
0
0
1
2
0
0
1
2
0
0
2
2
2
3
2
2
2
3
2
2
2
3
1
1
2
2
1
1
2
2
1
1
2
2
1
2
0
0
1
2
0
0
1
2
0
0
2
1
2
3
2
1
2
3
2
1
2
3
2
1
2
3
2
1
2
3
2
1
2
3
1
1
2
2
1
1
2
2
1
1
2
2
1
2
0
0
1
2
0
0
1
2
0
0
2
1
2
2
2
1
2
2
2
1
2
2


In [200]:
print(confusion_matrix(y_test,y_predicted))
print(classification_report(y_test,y_predicted))

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

